In [ ]:
get_ipython().system('token')

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `text2sql_deepseek` has been saved to /home/swsingh/.cache/huggingface/stored_tokens
Your token has been saved to /home/swsingh/.cache/huggingface/token
Login successful.
The current active token is: `text2sql_deepseek`


In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
import torch
import pandas as pd
import re
from tqdm import tqdm
import os

ModuleNotFoundError: No module named 'langchain'

In [2]:
# Optional: For better debugging info if CUDA errors occur
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [3]:
model_name = "deepseek-ai/deepseek-coder-33b-instruct"
custom_cache_dir = "/isilon/datalake/dma_research/scratch/damalab/Data/Swapnil/Personal/Model"  # Change this to your desired directory

# Load tokenizer with custom cache path
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=custom_cache_dir)

# Load model with custom cache path
model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    cache_dir=custom_cache_dir, 
    device_map={"":0}, 
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True
)

Loading checkpoint shards: 100%|██████████| 7/7 [00:28<00:00,  4.06s/it]


In [4]:
text_gen_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,  # or higher, like 1024 if needed
    temperature=0.3,
    top_p=0.9,
    do_sample=False, # Changed to deterministic decoding
    pad_token_id=tokenizer.eos_token_id
)

Device set to use cuda:0


In [5]:
# Initialize HuggingFacePipeline with the text generation pipeline
llm = HuggingFacePipeline(pipeline=text_gen_pipeline)

/scratch/swsingh/ipykernel_792359/2474907508.py:2: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=text_gen_pipeline)


In [6]:
# # Define a prompt template for SQL generation
# prompt = PromptTemplate(
#     input_variables=["text", "ini_query", "query"],
#     template="""
#     A SQL query was generated to answer the following natural language request:

#     Question: {text}
#     Initail SQL Querry:
#     ```sql
#     {ini_query}
#     ```
#     SQL Query:
#     ```sql
#     {query}
#     ```

#     Please do the following:
#     1. Verify if the SQL query correctly answers the question.
#     2. If it does, explain why it's correct.
#     3. If it's incorrect or suboptimal, explain the issue and provide a corrected or optimized version of the SQL query.
#     4. Make sure the syntax is valid and executable in standard SQL (PostgreSQL/MySQL-compatible).

#     Provide your answer in the following format:
#     - Verdict: Correct / Incorrect
#     - Explanation: <Your reasoning>
#     - Improved SQL (if applicable)/input SQL:
#     ```sql
#     <Improved query>
#     ```
#     """
# )

In [7]:
# # Create an LLMChain with the prompt template and language model
# llm_chain = LLMChain(prompt=prompt, llm=llm)

In [8]:
# # Example natural language query
# text = "How many pseudo users were active in the last 7 days but inactive in the last 2 days as of January 7, 2021?"
# db = "ga4"
# query = """
# SELECT COUNT(*) 
# FROM users 
# WHERE user_type = 'pseudo' 
# AND last_activity_date >= DATE_SUB('2021-01-07', INTERVAL 7 DAY) 
# AND last_activity_date < DATE_SUB('2021-01-07', INTERVAL 2 DAY)
# """
# # Generate the SQL statement
# sql_query = llm_chain.run(text=text, query=query)

# # Extract SQL block using regex
# sql_blocks = re.findall(r"```sql(.*?)```", sql_query, re.DOTALL)
# if sql_blocks:
#     cleaned_sql = sql_blocks[-1].strip()
#     print(cleaned_sql)
# else:
#     print("No SQL block found in response.")
# #print(sql_query)

In [29]:
import pandas as pd
df = pd.read_csv("refined_spider2_queries_cleaned_basic_final1.csv")
#df.head()
df

,instance_id,generated_sql,nl_query,db,external_knowledge,refined_sql,refined_sql_cleaned,final_query
0,bq001,"SELECT \n T1.visitor_id,\n T1.date_of_vi...",For each visitor who made at least one transac...,ga360,google_analytics_sample.ga_sessions.md,"SELECT \n T1.visitor_id,\n T1.date_of_vi...","SELECT T1.visitor_id, T1.date_of_visit, COUNT(...","SELECT \n v.visitor_id, \n MIN(v.date_of..."
1,bq002,"SELECT \r\n t1.source_name,\r\n MAX(t1.r...","During the first half of 2017, focusing on hi...",ga360,google_analytics_sample.ga_sessions.md,"SELECT \r\n t1.source_name,\r\n MAX(t1.r...","SELECT t1.source_name, MAX(t1.revenue) AS max_...","SELECT t1.source_name, MAX(t1.revenue) AS max_..."
2,bq003,"SELECT \n DATE_FORMAT(visits.visit_date, '%...","Between April 1 and July 31 of 2017, using the...",ga360,google_analytics_sample.ga_sessions.md,"SELECT \n DATE_FORMAT(visits.visit_date, '%...","SELECT DATE_FORMAT(visits.visit_date, '%Y-%m')...",SELECT \n DATE_FORMAT(visits.visit_date...
3,bq268,"SELECT \n MAX(DATEDIFF(day, MAX(last_visit)...",Identify the longest number of days between th...,ga360,NaN,"SELECT \n MAX(DATEDIFF(day, MAX(last_visit)...","SELECT MAX(DATEDIFF(day, MAX(last_visit), MIN(...","SELECT MAX(DATEDIFF(day, first_visit, last_eve..."
4,bq270,"SELECT \n p.category,\n COUNT(DISTINCT p...",What were the monthly add-to-cart and purchase...,ga360,ga360_hits.eCommerceAction.action_type.md,"SELECT \n p.category,\n COUNT(DISTINCT p...","SELECT p.category, COUNT(DISTINCT p.product_id...","SELECT p.category, COUNT(DISTINCT p.product_id..."
...,...,...,...,...,...,...,...,...
276,sf037,"SELECT \n StoreName,\n MIN(DISTANCE) AS ...",How can we find the shortest straight-line dis...,US_REAL_ESTATE,NaN,"SELECT \n StoreName,\n MIN(DISTANCE) AS ...","SELECT StoreName, MIN(DISTANCE) AS ShortestDis...",SELECT \n hd.StoreID AS TheHomeDepotSto...
277,sf029,"SELECT \r\n s.date AS date,\r\n s.asin A...",Generate a daily detailed sales report for eac...,AMAZON_VENDOR_ANALYTICS__SAMPLE_DATASET,NaN,"SELECT \r\n s.date AS date,\r\n s.asin A...","SELECT s.date AS date, s.asin AS asin, s.progr...","SELECT \n s.date, \n s.asin, \n ..."
278,sf013,"SELECT \n c.class_name AS Class,\n s.sub...",Compare the total road lengths in Amsterdam an...,NETHERLANDS_OPEN_MAP_DATA,NaN,"SELECT \n c.class_name AS Class,\n s.sub...","SELECT c.class_name AS Class, s.sub_class_name...","SELECT c.class_name AS Class, s.sub_class_name..."
279,sf041,"SELECT \r\n T1.date AS date,\r\n T1.hour...","Produce a report for ERCOT on October 1, 2022,...",YES_ENERGY__SAMPLE_DATA,ERCOT_Daily_Market_Dynamics_Report.md,# Refine the following SQL query using externa...,# Refine the following SQL query using externa...,<Improved query>


In [30]:
df1 = df[df['final_query']=='<Improved query>']
df1.shape

(39, 8)

In [31]:
# import os
# import glob

# external_knowledge = {}
# external_knowledge_dir = '/isilon/datalake/dma_research/scratch/damalab/Data/Swapnil/Personal/Text2SQL/Spider2/spider2-lite/resource/documents/'

# # Load each Markdown (.md) file into the dictionary
# for file_path in glob.glob(os.path.join(external_knowledge_dir, '*.md')):
#     query_id = os.path.basename(file_path)
#     with open(file_path, 'r', encoding='utf-8') as file:
#         external_knowledge[query_id] = file.read().strip()

In [32]:
prompts = []
for i in range(df1.shape[0]):
    text = df1['nl_query'].iloc[i]
    query = df1['refined_sql_cleaned'].iloc[i]
    db = df1['db'].iloc[i]
    # external_knowledge_name = df['external_knowledge'].iloc[i]

    # # Check for NaN or missing knowledge
    # if pd.isna(external_knowledge_name) or external_knowledge_name not in external_knowledge:
    #     ek = "N/A"
    # else:
    #     ek = external_knowledge[external_knowledge_name]
    # db = df['db'].iloc[i]

    prompt = f"""
    A SQL query was generated to answer the following natural language request:

    Question: {text}
    Database: {db}
    SQL Query:
    ```sql
    {query}
    ```

    Please do the following:
    1. Verify if the SQL query correctly answers the question.
    2. If it does, explain why it's correct.
    3. If it's incorrect or suboptimal, explain the issue and provide a corrected or optimized version of the SQL query.
    4. Make sure the syntax is valid and executable in standard SQL (PostgreSQL/MySQL-compatible).

    Provide your answer in the following format:
    - Improved SQL (if applicable)/input SQL:
    ```sql
    <Improved query>
    ```
    """
    prompts.append(prompt.strip())

In [33]:
from math import ceil

In [34]:
batch_size = 1  # Choose based on your GPU memory
final_query = []
outputs = []

for i in tqdm(range(0, len(prompts), batch_size)):
    batch_prompts = prompts[i:i + batch_size]
    
    # Run batch through the Hugging Face pipeline
    responses = text_gen_pipeline(batch_prompts)
    
    for r in responses:
        output = r[0]['generated_text']
        sql_blocks = re.findall(r"```sql(.*?)```", output, re.DOTALL)
        cleaned_sql = sql_blocks[-1].strip() if sql_blocks else ""
        final_query.append(cleaned_sql)
        outputs.append(output)

  0%|          | 0/39 [00:00<?, ?it/s]/isilon/datalake/dma_research/scratch/damalab/Data/Swapnil/miniconda3/envs/monai_conda/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/isilon/datalake/dma_research/scratch/damalab/Data/Swapnil/miniconda3/envs/monai_conda/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
100%|██████████| 39/39 [46:27<00:00, 71.49s/it]


In [35]:
outputs[-1]

"A SQL query was generated to answer the following natural language request:\n\n    Question: Produce a report for ERCOT on October 1, 2022, that combines hourly data on day-ahead and real-time prices from node ID 10000697078, load forecasts (datatypeid 19060) and actual loads, plus wind (forecast datatypeid 9285, actual datatypeid 16) and solar (forecast datatypeid 662, actual datatypeid 650) generation forecasts and actuals from object ID 10000712973. This report should include time zone alignments, peak classifications, and net load calculations, providing insights into daily operational dynamics and efficiency.\n    Database: YES_ENERGY__SAMPLE_DATA\n    SQL Query:\n    ```sql\n    # Refine the following SQL query using external knowledge and schema context # Fix syntax errors and optimize for clarity and correctness Database: YES_ENERGY__SAMPLE_DATA External knowledge: ERCOT_Daily_Market_Dynamics_Report.md Natural language query: Produce a report for ERCOT on October 1, 2022, that

In [36]:
df1['final_query'] = final_query

/scratch/swsingh/ipykernel_792359/2912785296.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['final_query'] = final_query


In [37]:
df1

,instance_id,generated_sql,nl_query,db,external_knowledge,refined_sql,refined_sql_cleaned,final_query
15,sf_bq052,"SELECT \n p.id AS patent_id,\n p.title A...",Retrieve the following information for U.S. pa...,PATENTSVIEW,NaN,"SELECT \n p.id AS patent_id,\n p.title A...","SELECT p.id AS patent_id, p.title AS patent_ti...","SELECT p.id AS patent_id, p.title AS patent_ti..."
17,sf_bq101,"SELECT \n package_name,\n COUNT(*) AS oc...","From GitHub Repos contents, how can we identif...",GITHUB_REPOS,NaN,"SELECT \n package_name,\n COUNT(*) AS oc...","SELECT package_name, COUNT(*) AS occurrence_co...",<Improved query>
33,bq031,"SELECT \n DATE_TRUNC('day', timestamp) AS d...",Provide the daily weather data for Rochester f...,noaa_data,NaN,"SELECT \n date,\n temperature,\n prec...","SELECT date, temperature, precipitation, wind_...","SELECT date, temperature, precipitation, wind_..."
39,bq293,"SELECT \r\n zip_code, \r\n COUNT(*) AS t...",I want to analyze New York City yellow taxi tr...,new_york_geo,functions_st_contains.md,# Refine the following SQL query using externa...,# Refine the following SQL query using externa...,<Improved query>
40,sf_bq131,SELECT statement with the COUNT function to co...,What is the number of bus stops for the bus ne...,GEO_OPENSTREETMAP,functions_st_dwithin.md,SELECT COUNT(*) \nFROM (\n SELECT \n STOP_...,"SELECT COUNT(*) FROM ( SELECT STOP_ID, NAME, S...",<Improved query>
43,sf_bq410,"SELECT`, `FROM`, `WHERE`, `GROUP BY`, `ORDER B...",Find the top 3 states with the smallest adjust...,CENSUS_BUREAU_ACS_2,NaN,"SELECT\n STATE_NAME,\n AGGREGATE_VALUE,\...","SELECT STATE_NAME, AGGREGATE_VALUE, TRACT_CODE...","SELECT \n STATE_NAME, \n SUM(CAS..."
44,sf_bq253,"SELECT \n r.name AS relation_name,\n f.f...",Find the name of the OpenStreetMap relation th...,GEO_OPENSTREETMAP,functions_st_dwithin.md,# Refine the following SQL query using externa...,# Refine the following SQL query using externa...,SELECT relation_name\n FROM planet_features...
48,sf_bq250,"SELECT \r\n g.id AS grid_id,\r\n g.name ...",Based on the most recent 1km population grid d...,GEO_OPENSTREETMAP_WORLDPOP,OpenStreetMap_data_in_layered_GIS_format.md,"SELECT \r\n g.id AS grid_id,\r\n g.name ...","SELECT g.id AS grid_id, g.name AS grid_name, s...","SELECT g.id AS grid_id, g.name AS grid_name, s..."
50,sf_bq342,SELECT AVG(transaction_value) AS avg_change\r\...,What is the difference between the average hou...,CRYPTO,NaN,SELECT AVG(transaction_value) AS avg_change\r\...,SELECT AVG(transaction_value) AS avg_change FR...,<Improved query>
51,sf_bq444,"SELECT \n t.block_timestamp,\n t.transac...","Can you pull the blockchain timestamp, block n...",CRYPTO,ethereum_logs_and_events_overview.md,"SELECT \n t.block_timestamp,\n t.transac...","SELECT t.block_timestamp, t.transaction_hash, ...","SELECT t.block_timestamp, t.transaction_hash, ..."


In [38]:
df1.to_csv('refined_spider2_queries_cleaned_basic_final2.csv', index=False)

In [21]:
# final_query = []
# for i in tqdm(range(df.shape[0])):
#     text = df['nl_query'].iloc[i]
#     query = df['refined_sql_cleaned'].iloc[i]
#     sql_query = llm_chain.run({"text": text, "query": query})
#     sql_blocks = re.findall(r"```sql(.*?)```", sql_query, re.DOTALL)
#     cleaned_sql = sql_blocks[-1].strip()

#     # try:
#     #     sql_query = llm_chain.run({"text": text, "query": query})
#     #     sql_blocks = re.findall(r"```sql(.*?)```", sql_query, re.DOTALL)
#     #     cleaned_sql = sql_blocks[-1].strip() if sql_blocks else ""
#     # except Exception as e:
#     #     print(f"Error at index {i}: {e}")
#     #     cleaned_sql = ""
    
#     final_query.append(cleaned_sql)

In [14]:
len(final_query)

547

In [15]:
df['final_query'] = final_query

In [16]:
df

,instance_id,generated_sql,nl_query,db,external_knowledge,refined_sql,refined_sql_cleaned,final_query
0,bq011,"create the appropriate SQL query.\n\nHowever, ...",How many distinct pseudo users had positive en...,ga4,NaN,"SELECT \n DATE_TRUNC('month', date) AS mont...","SELECT DATE_TRUNC('month', date) AS month, COU...",<Improved query>
1,bq010,"SELECT \n Product,\n SUM(Quantity * Reve...",Find the top-selling product among customers w...,ga360,google_analytics_sample.ga_sessions.md,"SELECT \n Product,\n SUM(Quantity * Reve...","SELECT Product, SUM(Quantity * Revenue) AS Tot...","SELECT Product, SUM(Quantity * Revenue) AS Tot..."
2,bq009,"SELECT t1.total_transaction_revenue, t1.monthl...",Which traffic source has the highest total tra...,ga360,google_analytics_sample.ga_sessions.md,"SELECT t1.traffic_source, t1.total_transaction...","SELECT t1.traffic_source, t1.total_transaction...","SELECT traffic_source, MAX(total_transaction_r..."
3,bq001,"SELECT \n T1.visitor_id,\n T1.date_of_vi...",For each visitor who made at least one transac...,ga360,google_analytics_sample.ga_sessions.md,"SELECT \n T1.visitor_id,\n T1.date_of_vi...","SELECT T1.visitor_id, T1.date_of_visit, COUNT(...",<Improved query>
4,bq002,"SELECT \r\n t1.source_name,\r\n MAX(t1.r...","During the first half of 2017, focusing on hi...",ga360,google_analytics_sample.ga_sessions.md,"SELECT \r\n t1.source_name,\r\n MAX(t1.r...","SELECT t1.source_name, MAX(t1.revenue) AS max_...","SELECT trafficSource.source AS source_name, MA..."
...,...,...,...,...,...,...,...,...
542,sf009,SELECT \n b.building_class AS 'Building Typ...,A real estate company needs a detailed side-by...,NETHERLANDS_OPEN_MAP_DATA,NaN,SELECT \n b.building_class AS 'Building Typ...,"SELECT b.building_class AS 'Building Type', b....",<Improved query>
543,sf013,"SELECT \n c.class_name AS Class,\n s.sub...",Compare the total road lengths in Amsterdam an...,NETHERLANDS_OPEN_MAP_DATA,NaN,"SELECT \n c.class_name AS Class,\n s.sub...","SELECT c.class_name AS Class, s.sub_class_name...",<Improved query>
544,sf041,"SELECT \r\n T1.date AS date,\r\n T1.hour...","Produce a report for ERCOT on October 1, 2022,...",YES_ENERGY__SAMPLE_DATA,ERCOT_Daily_Market_Dynamics_Report.md,# Refine the following SQL query using externa...,# Refine the following SQL query using externa...,<Improved query>
545,sf011,"SELECT \n census_value,\n CASE \n ...",Determine the population distribution within e...,CENSUS_GALAXY__ZIP_CODE_TO_BLOCK_GROUP_SAMPLE,NaN,"SELECT \n census_value,\n CASE \n ...","SELECT census_value, CASE WHEN state_county_tr...",<Improved query>


In [18]:
df[df['final_query']=='<Improved query>']

,instance_id,generated_sql,nl_query,db,external_knowledge,refined_sql,refined_sql_cleaned,final_query
0,bq011,"create the appropriate SQL query.\n\nHowever, ...",How many distinct pseudo users had positive en...,ga4,NaN,"SELECT \n DATE_TRUNC('month', date) AS mont...","SELECT DATE_TRUNC('month', date) AS month, COU...",<Improved query>
3,bq001,"SELECT \n T1.visitor_id,\n T1.date_of_vi...",For each visitor who made at least one transac...,ga360,google_analytics_sample.ga_sessions.md,"SELECT \n T1.visitor_id,\n T1.date_of_vi...","SELECT T1.visitor_id, T1.date_of_visit, COUNT(...",<Improved query>
5,bq003,"SELECT \n DATE_FORMAT(visits.visit_date, '%...","Between April 1 and July 31 of 2017, using the...",ga360,google_analytics_sample.ga_sessions.md,"SELECT \n DATE_FORMAT(visits.visit_date, '%...","SELECT DATE_FORMAT(visits.visit_date, '%Y-%m')...",<Improved query>
7,bq008,SELECT \n MAX(`visit_time`) AS `most_visite...,"In January 2017, among visitors whose campaign...",ga360,google_analytics_sample.ga_sessions.md,SELECT \n MAX(`visit_time`) AS `most_visite...,SELECT MAX(`visit_time`) AS `most_visited_page...,<Improved query>
9,bq268,"SELECT \n MAX(DATEDIFF(day, MAX(last_visit)...",Identify the longest number of days between th...,ga360,NaN,"SELECT \n MAX(DATEDIFF(day, MAX(last_visit)...","SELECT MAX(DATEDIFF(day, MAX(last_visit), MIN(...",<Improved query>
...,...,...,...,...,...,...,...,...
540,sf029,"SELECT \r\n s.date AS date,\r\n s.asin A...",Generate a daily detailed sales report for eac...,AMAZON_VENDOR_ANALYTICS__SAMPLE_DATASET,NaN,"SELECT \r\n s.date AS date,\r\n s.asin A...","SELECT s.date AS date, s.asin AS asin, s.progr...",<Improved query>
542,sf009,SELECT \n b.building_class AS 'Building Typ...,A real estate company needs a detailed side-by...,NETHERLANDS_OPEN_MAP_DATA,NaN,SELECT \n b.building_class AS 'Building Typ...,"SELECT b.building_class AS 'Building Type', b....",<Improved query>
543,sf013,"SELECT \n c.class_name AS Class,\n s.sub...",Compare the total road lengths in Amsterdam an...,NETHERLANDS_OPEN_MAP_DATA,NaN,"SELECT \n c.class_name AS Class,\n s.sub...","SELECT c.class_name AS Class, s.sub_class_name...",<Improved query>
544,sf041,"SELECT \r\n T1.date AS date,\r\n T1.hour...","Produce a report for ERCOT on October 1, 2022,...",YES_ENERGY__SAMPLE_DATA,ERCOT_Daily_Market_Dynamics_Report.md,# Refine the following SQL query using externa...,# Refine the following SQL query using externa...,<Improved query>


In [17]:
df.to_csv('refined_spider2_queries_cleaned_basic_final.csv', index=False)

In [3]:
df = pd.read_csv("refined_spider2_queries_cleaned_basic_final.csv")
df1 = pd.read_csv("refined_spider2_queries_cleaned_basic_final1.csv")
df2 = pd.read_csv("refined_spider2_queries_cleaned_basic_final2.csv")

In [4]:
# Copy original df
final_df = df.copy()

# Identify unresolved rows in df
unresolved_mask_df = final_df["final_query"].isna() | (final_df["final_query"] == "<Improved query>")

# Fill those unresolved rows with df1 values
final_df.loc[unresolved_mask_df, "final_query"] = df1["final_query"].values

# Re-identify unresolved rows after df1 patch
unresolved_mask_df1 = final_df["final_query"].isna() | (final_df["final_query"] == "<Improved query>")

# Fill those with df2 values
final_df.loc[unresolved_mask_df1, "final_query"] = df2["final_query"].values

In [5]:
final_df.to_csv("refined_spider2_queries_cleaned_basic_final_final.csv", index=False)

In [6]:
df = pd.read_csv("refined_spider_queries_cleaned_basic_final.csv")
df1 = pd.read_csv("refined_spider_queries_cleaned_basic_final1.csv")
df2 = pd.read_csv("refined_spider_queries_cleaned_basic_final2.csv")
# Copy original df
final_df = df.copy()

# Identify unresolved rows in df
unresolved_mask_df = final_df["final_query"].isna() | (final_df["final_query"] == "<Improved query>")

# Fill those unresolved rows with df1 values
final_df.loc[unresolved_mask_df, "final_query"] = df1["final_query"].values

# Re-identify unresolved rows after df1 patch
unresolved_mask_df1 = final_df["final_query"].isna() | (final_df["final_query"] == "<Improved query>")

# Fill those with df2 values
final_df.loc[unresolved_mask_df1, "final_query"] = df2["final_query"].values
final_df.to_csv("refined_spider_queries_cleaned_basic_final_final.csv", index=False)

In [2]:
df = pd.read_csv('final_refined_partial_30.csv')
df.head()

,instance_id,question,db,initial_sql,refined_sql,final_query
0,bq011,How many pseudo users were active in the last ...,ga4,Generate an SQL query using the database name ...,SELECT COUNT(*) \nFROM users \nWHERE last_acti...,SELECT COUNT(*) \n FROM users \n WHERE l...
1,bq010,Find the top-selling product among customers w...,ga360,Generate an SQL query using the database name ...,"SELECT \n s.product_name,\n s.sale_price...","SELECT \n s.product_name,\n s.sa..."
2,bq009,Which traffic source has the highest total tra...,ga360,Generate an SQL query using the database name ...,SELECT \n t1TrafficSourceName AS TrafficSou...,"SELECT \n t1.TrafficSourceName,\n ..."
3,bq001,For each visitor who made at least one transac...,ga360,Generate an SQL query using the database name ...,"SELECT \n DATE_FORMAT(first_visit_date, '%Y...","SELECT \n DATE_FORMAT(first_visit_date,..."
4,bq002,"During the first half of 2017, focusing on hi...",ga360,Generate an SQL query using the database name ...,"SELECT \r\n p.product_name,\r\n MAX(p.re...","SELECT \n p.product_name,\n MAX(..."


In [3]:
df.db.value_counts()

db
PATENTS           13
ga360             12
PATENTS_GOOGLE     4
ga4                1
Name: count, dtype: int64

In [5]:
from google.oauth2 import service_account
from google.cloud import bigquery

credential_path = '/isilon/datalake/dma_research/scratch/damalab/Data/Swapnil/Personal/Spider2/spider2-lite/evaluation_suite/bigquery_credential_swapnil.json' # path/to/your/keyfile.json
credentials = service_account.Credentials.from_service_account_file(credential_path)
client = bigquery.Client(credentials=credentials)

# alternatively, you can also set the credential path via environment vairable
# import os
# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/path/to/keyfile.json"
# client = bigquery.Client()

# Perform a sample query.
sql_query = 'SELECT name FROM `bigquery-public-data.usa_names.usa_1910_2013` WHERE state = "TX" LIMIT 10'
query_job = client.query(sql_query)  # API request
rows = query_job.result()  # Waits for query to finish

for row in rows:
    print(row.name)

Bonnie
Manuela
Florence
Leona
Agnes
Nettie
Elvira
Sadie
Annie
Mozelle


In [7]:
df.head()

,instance_id,question,db,initial_sql,refined_sql,final_query
0,bq011,How many pseudo users were active in the last ...,ga4,Generate an SQL query using the database name ...,SELECT COUNT(*) \nFROM users \nWHERE last_acti...,SELECT COUNT(*) \n FROM users \n WHERE l...
1,bq010,Find the top-selling product among customers w...,ga360,Generate an SQL query using the database name ...,"SELECT \n s.product_name,\n s.sale_price...","SELECT \n s.product_name,\n s.sa..."
2,bq009,Which traffic source has the highest total tra...,ga360,Generate an SQL query using the database name ...,SELECT \n t1TrafficSourceName AS TrafficSou...,"SELECT \n t1.TrafficSourceName,\n ..."
3,bq001,For each visitor who made at least one transac...,ga360,Generate an SQL query using the database name ...,"SELECT \n DATE_FORMAT(first_visit_date, '%Y...","SELECT \n DATE_FORMAT(first_visit_date,..."
4,bq002,"During the first half of 2017, focusing on hi...",ga360,Generate an SQL query using the database name ...,"SELECT \r\n p.product_name,\r\n MAX(p.re...","SELECT \n p.product_name,\n MAX(..."


In [11]:
df['initial_sql'][0]

"Generate an SQL query using the database name ga4 for the following question: How many pseudo users were active in the last 7 days but inactive in the last 2 days as of January 7, 2021? Additionally, include a condition to filter out users who have been banned within the last 3 months. The table name is 'users' and it contains columns 'username', 'pseudo_user_id', 'last_activity_date', 'banned_date'. Ensure your query includes JOIN clauses with appropriate conditions.\nSELECT COUNT(*) \nFROM users \nWHERE last_activity_date >= DATE_SUB(CURDATE(), INTERVAL 7 DAY) AND last_activity_date <= DATE_SUB(CURDATE(), INTERVAL 2 DAY) \nAND banned_date IS NULL\nAND username NOT IN (\n    SELECT username \n    FROM users \n    WHERE last_activity_date >= DATE_SUB(CURDATE(), INTERVAL 3 MONTH)\n); \n\nExplanation:\n- The first query selects all rows where the `last_activity_date` is greater than or equal to the current date minus seven days and less than the current date plus two days.\n- The second

In [12]:
import os
import glob

In [13]:
ground_truth_queries = {}

ground_truth_dir = '/isilon/datalake/dma_research/scratch/damalab/Data/Swapnil/Personal/Text2SQL/Spider2/spider2-lite/evaluation_suite/gold/sql'

# Load each ground truth SQL file into the dictionary
for file_path in glob.glob(os.path.join(ground_truth_dir, '*.sql')):
    query_id = os.path.splitext(os.path.basename(file_path))[0]
    with open(file_path, 'r', encoding='utf-8') as file:
        ground_truth_queries[query_id] = file.read().strip()

In [15]:
# Initialize counters for total and matched queries
total_queries = 0
exact_matches = 0

In [16]:
df.head()

,instance_id,question,db,initial_sql,refined_sql,final_query
0,bq011,How many pseudo users were active in the last ...,ga4,Generate an SQL query using the database name ...,SELECT COUNT(*) \nFROM users \nWHERE last_acti...,SELECT COUNT(*) \n FROM users \n WHERE l...
1,bq010,Find the top-selling product among customers w...,ga360,Generate an SQL query using the database name ...,"SELECT \n s.product_name,\n s.sale_price...","SELECT \n s.product_name,\n s.sa..."
2,bq009,Which traffic source has the highest total tra...,ga360,Generate an SQL query using the database name ...,SELECT \n t1TrafficSourceName AS TrafficSou...,"SELECT \n t1.TrafficSourceName,\n ..."
3,bq001,For each visitor who made at least one transac...,ga360,Generate an SQL query using the database name ...,"SELECT \n DATE_FORMAT(first_visit_date, '%Y...","SELECT \n DATE_FORMAT(first_visit_date,..."
4,bq002,"During the first half of 2017, focusing on hi...",ga360,Generate an SQL query using the database name ...,"SELECT \r\n p.product_name,\r\n MAX(p.re...","SELECT \n p.product_name,\n MAX(..."


In [19]:
CLAUSE_KEYWORDS = ['select', 'from', 'where', 'group by', 'order by', 'having', 'limit', 'join']

def extract_structure(sql):
    sql = sql.lower()
    structure = []

    for keyword in CLAUSE_KEYWORDS:
        # Add to structure if keyword exists in SQL
        if re.search(r'\b' + keyword + r'\b', sql):
            structure.append(keyword)
    return structure

# Normalize and extract structure
def normalize_structure(query):
    if not isinstance(query, str): return []
    return extract_structure(query)

In [20]:
matches = 0
total = len(df)

for _, row in df.iterrows():
    query_id = row['instance_id']
    pred_struct = normalize_structure(row['final_query'])
    true_struct = normalize_structure(ground_truth_queries.get(query_id, ''))

    if pred_struct == true_struct:
        matches += 1

In [21]:
score = matches / total if total else 0
print(f'Structural Exact Match Score: {score:.2%}')

Structural Exact Match Score: 20.00%


In [22]:
matches

6

# evaluation

In [10]:
import pandas as pd
import os
import glob
import re

## spider 2

In [239]:
CLAUSE_KEYWORDS = ['select', 'from', 'where', 'group by', 'order by', 'having', 'limit', 'join']
COMPONENTS_1 = {'where', 'group by', 'order by', 'limit', 'join', 'or', 'like', 'having'}
COMPONENTS_2 = {'except', 'union', 'intersect', 'in', 'not in', 'nested'}

# ----------------------------
# Structure Matching
# ----------------------------
def extract_structure(sql):
    sql = sql.lower()
    return [kw for kw in CLAUSE_KEYWORDS if kw in sql]

def normalize_structure(query):
    if not isinstance(query, str):
        return []
    return extract_structure(query)

# ----------------------------
# Hardness Classification
# ----------------------------
def extract_sql_metadata(sql):
    sql = sql.lower()
    agg_count = len(re.findall(r'\b(sum|count|max|min|avg)\(', sql))

    select_cols = re.findall(r'select (.*?) from', sql)
    select_count = len(select_cols[0].split(',')) if select_cols else 0

    where_clause = re.search(r'where (.*?)(group by|order by|having|limit|$)', sql)
    where_count = len(re.split(r'\s+and\s+|\s+or\s+', where_clause.group(1))) if where_clause else 0

    group_by_clause = re.search(r'group by (.*?)(order by|having|limit|$)', sql)
    group_by_count = len(group_by_clause.group(1).split(',')) if group_by_clause else 0

    components_1 = {k for k in COMPONENTS_1 if k in sql}

    components_2 = set()
    if re.search(r'\bexcept\b', sql): components_2.add('except')
    if re.search(r'\bunion\b', sql): components_2.add('union')
    if re.search(r'\bintersect\b', sql): components_2.add('intersect')
    if re.search(r'\b(not\s+)?in\s*\(', sql): components_2.add('in')
    if re.search(r'\(\s*select\b', sql): components_2.add('nested')

    return {
        'agg_count': agg_count,
        'select_count': select_count,
        'where_count': where_count,
        'group_by_count': group_by_count,
        'components_1': components_1,
        'components_2': components_2
    }

def classify_sql_hardness(info):
    c1 = len(info['components_1'])
    c2 = len(info['components_2'])
    other = sum([
        info['agg_count'] > 1,
        info['select_count'] > 1,
        info['where_count'] > 1,
        info['group_by_count'] > 1
    ])

    if c2 == 0 and c1 <= 1 and other == 0:
        return 'easy'
    if c2 == 0 and ((c1 == 1 and other <= 2) or (c1 == 2 and other < 2)):
        return 'medium'
    if c2 == 0 and ((c1 <= 2 and other > 2) or (2 < c1 <= 3 and other <= 2)):
        return 'hard'
    if c2 == 1 and c1 <= 1 and other == 0:
        return 'hard'
    return 'extra'

In [248]:
# Load ground truth SQLs
ground_truth_dir = 'spider2_lite_gold/sql'
ground_truth_queries = {}

for file_path in glob.glob(os.path.join(ground_truth_dir, '*.sql')):
    query_id = os.path.splitext(os.path.basename(file_path))[0]
    with open(file_path, 'r', encoding='utf-8') as f:
        ground_truth_queries[query_id] = f.read().strip()

# Load predictions
df = pd.read_csv('refined_spider2_queries_cleaned_basic_final_final.csv')

# Evaluate
results = []

for _, row in df.iterrows():
    query_id = row['instance_id']
    pred_sql = row['final_query']
    gold_sql = ground_truth_queries.get(query_id, '')

    # Structure match
    pred_struct = normalize_structure(pred_sql)
    gold_struct = normalize_structure(gold_sql)

    exact_match = int(pred_struct == gold_struct)
    jaccard = len(set(pred_struct) & set(gold_struct)) / len(set(pred_struct) | set(gold_struct)) if (pred_struct or gold_struct) else 1.0

    # Hardness
    hardness = classify_sql_hardness(extract_sql_metadata(gold_sql))

    results.append({
        'instance_id': query_id,
        'exact_match': exact_match,
        'jaccard_score': jaccard,
        'hardness': hardness
    })

# ----------------------------
# Results
# ----------------------------
results_df = pd.DataFrame(results)

print(f"✅ Exact Match Score: {results_df['exact_match'].mean():.2%}")
print(f"✅ Avg Jaccard Structural Score: {results_df['jaccard_score'].mean():.2%}")

print("\n📊 Count of Queries by Hardness:")
print(results_df['hardness'].value_counts().sort_index())

print("\n📊 Exact Match by Hardness:")
print(results_df.groupby('hardness')['exact_match'].mean())

print("\n📊 Jaccard Score by Hardness:")
print(results_df.groupby('hardness')['jaccard_score'].mean())


✅ Exact Match Score: 10.24%
✅ Avg Jaccard Structural Score: 37.04%

📊 Count of Queries by Hardness:
hardness
easy      291
extra     253
hard        2
medium      1
Name: count, dtype: int64

📊 Exact Match by Hardness:
hardness
easy      0.000000
extra     0.217391
hard      0.500000
medium    0.000000
Name: exact_match, dtype: float64

📊 Jaccard Score by Hardness:
hardness
easy      0.000000
extra     0.792979
hard      0.750000
medium    0.500000
Name: jaccard_score, dtype: float64


### spider 2.0 ablation qwen 

In [212]:
# Load ground truth SQLs
ground_truth_dir = 'spider2_lite_gold/sql'
ground_truth_queries = {}

for file_path in glob.glob(os.path.join(ground_truth_dir, '*.sql')):
    query_id = os.path.splitext(os.path.basename(file_path))[0]
    with open(file_path, 'r', encoding='utf-8') as f:
        ground_truth_queries[query_id] = f.read().strip()

# Load predictions
df = pd.read_csv('refined_spider2_queries_cleaned_basic_final_final.csv')

# Evaluate
results = []

for _, row in df.iterrows():
    query_id = row['instance_id']
    pred_sql = row['generated_sql']
    gold_sql = ground_truth_queries.get(query_id, '')

    # Structure match
    pred_struct = normalize_structure(pred_sql)
    gold_struct = normalize_structure(gold_sql)

    exact_match = int(pred_struct == gold_struct)
    jaccard = len(set(pred_struct) & set(gold_struct)) / len(set(pred_struct) | set(gold_struct)) if (pred_struct or gold_struct) else 1.0

    # Hardness
    hardness = classify_sql_hardness(extract_sql_metadata(gold_sql))

    results.append({
        'instance_id': query_id,
        'exact_match': exact_match,
        'jaccard_score': jaccard,
        'hardness': hardness
    })

# ----------------------------
# Results
# ----------------------------
results_df = pd.DataFrame(results)

print(f"✅ Exact Match Score: {results_df['exact_match'].mean():.2%}")
print(f"✅ Avg Jaccard Structural Score: {results_df['jaccard_score'].mean():.2%}")

print("\n📊 Count of Queries by Hardness:")
print(results_df['hardness'].value_counts().sort_index())

print("\n📊 Exact Match by Hardness:")
print(results_df.groupby('hardness')['exact_match'].mean())

print("\n📊 Jaccard Score by Hardness:")
print(results_df.groupby('hardness')['jaccard_score'].mean())


✅ Exact Match Score: 7.31%
✅ Avg Jaccard Structural Score: 33.77%

📊 Count of Queries by Hardness:
hardness
easy      291
extra     253
hard        2
medium      1
Name: count, dtype: int64

📊 Exact Match by Hardness:
hardness
easy      0.027491
extra     0.122530
hard      0.500000
medium    0.000000
Name: exact_match, dtype: float64

📊 Jaccard Score by Hardness:
hardness
easy      0.027491
extra     0.690589
hard      0.750000
medium    0.500000
Name: jaccard_score, dtype: float64


### spider 2.0 ablation starcoder 

In [213]:
# Load ground truth SQLs
ground_truth_dir = 'spider2_lite_gold/sql'
ground_truth_queries = {}

for file_path in glob.glob(os.path.join(ground_truth_dir, '*.sql')):
    query_id = os.path.splitext(os.path.basename(file_path))[0]
    with open(file_path, 'r', encoding='utf-8') as f:
        ground_truth_queries[query_id] = f.read().strip()

# Load predictions
df = pd.read_csv('refined_spider2_queries_cleaned_basic_final_final.csv')

# Evaluate
results = []

for _, row in df.iterrows():
    query_id = row['instance_id']
    pred_sql = row['refined_sql_cleaned']
    gold_sql = ground_truth_queries.get(query_id, '')

    # Structure match
    pred_struct = normalize_structure(pred_sql)
    gold_struct = normalize_structure(gold_sql)

    exact_match = int(pred_struct == gold_struct)
    jaccard = len(set(pred_struct) & set(gold_struct)) / len(set(pred_struct) | set(gold_struct)) if (pred_struct or gold_struct) else 1.0

    # Hardness
    hardness = classify_sql_hardness(extract_sql_metadata(gold_sql))

    results.append({
        'instance_id': query_id,
        'exact_match': exact_match,
        'jaccard_score': jaccard,
        'hardness': hardness
    })

# ----------------------------
# Results
# ----------------------------
results_df = pd.DataFrame(results)

print(f"✅ Exact Match Score: {results_df['exact_match'].mean():.2%}")
print(f"✅ Avg Jaccard Structural Score: {results_df['jaccard_score'].mean():.2%}")

print("\n📊 Count of Queries by Hardness:")
print(results_df['hardness'].value_counts().sort_index())

print("\n📊 Exact Match by Hardness:")
print(results_df.groupby('hardness')['exact_match'].mean())

print("\n📊 Jaccard Score by Hardness:")
print(results_df.groupby('hardness')['jaccard_score'].mean())


✅ Exact Match Score: 8.41%
✅ Avg Jaccard Structural Score: 33.97%

📊 Count of Queries by Hardness:
hardness
easy      291
extra     253
hard        2
medium      1
Name: count, dtype: int64

📊 Exact Match by Hardness:
hardness
easy      0.061856
extra     0.106719
hard      0.500000
medium    0.000000
Name: exact_match, dtype: float64

📊 Jaccard Score by Hardness:
hardness
easy      0.061856
extra     0.655599
hard      0.714286
medium    0.500000
Name: jaccard_score, dtype: float64


## spider

In [246]:
# Load ground truth SQLs
df1 = pd.read_parquet('spider.parquet')

# Load predictions
df = pd.read_csv('refined_spider_queries_cleaned_basic_final_final.csv')

df2 = pd.merge(df, df1, left_on=['nl_query', 'db'], right_on=['question', 'db_id'], how='inner')
df.dropna(inplace=True)
df2.shape

results = []

for _, row in df2.iterrows():
    query_id = row['instance_id']
    pred_sql = row['final_query']
    gold_sql = row['query']

    # Structure match
    pred_struct = normalize_structure(pred_sql)
    gold_struct = normalize_structure(gold_sql)

    exact_match = int(pred_struct == gold_struct)
    jaccard = len(set(pred_struct) & set(gold_struct)) / len(set(pred_struct) | set(gold_struct)) if (pred_struct or gold_struct) else 1.0

    # Hardness
    hardness = classify_sql_hardness(extract_sql_metadata(gold_sql))

    results.append({
        'instance_id': query_id,
        'exact_match': exact_match,
        'jaccard_score': jaccard,
        'hardness': hardness
    })

# ----------------------------
# Results
# ----------------------------
results_df = pd.DataFrame(results)

print(f"✅ Exact Match Score: {results_df['exact_match'].mean():.2%}")
print(f"✅ Avg Jaccard Structural Score: {results_df['jaccard_score'].mean():.2%}")

print("\n📊 Count of Queries by Hardness:")
print(results_df['hardness'].value_counts().sort_index())

print("\n📊 Exact Match by Hardness:")
print(results_df.groupby('hardness')['exact_match'].mean())

print("\n📊 Jaccard Score by Hardness:")
print(results_df.groupby('hardness')['jaccard_score'].mean())

✅ Exact Match Score: 34.04%
✅ Avg Jaccard Structural Score: 76.98%

📊 Count of Queries by Hardness:
hardness
easy      1220
extra     2081
hard      1403
medium    2426
Name: count, dtype: int64

📊 Exact Match by Hardness:
hardness
easy      0.574590
extra     0.322441
hard      0.225232
medium    0.304617
Name: exact_match, dtype: float64

📊 Jaccard Score by Hardness:
hardness
easy      0.838996
extra     0.767738
hard      0.725476
medium    0.762382
Name: jaccard_score, dtype: float64


### spider ablation qwen

In [247]:
# Load ground truth SQLs
df1 = pd.read_parquet('spider.parquet')

# Load predictions
df = pd.read_csv('refined_spider_queries_cleaned_basic_final_final.csv')

df2 = pd.merge(df, df1, left_on=['nl_query', 'db'], right_on=['question', 'db_id'], how='inner')
df.dropna(inplace=True)
df2.shape

results = []

for _, row in df2.iterrows():
    query_id = row['instance_id']
    pred_sql = row['generated_sql']
    gold_sql = row['query']

    # Structure match
    pred_struct = normalize_structure(pred_sql)
    gold_struct = normalize_structure(gold_sql)

    exact_match = int(pred_struct == gold_struct)
    jaccard = len(set(pred_struct) & set(gold_struct)) / len(set(pred_struct) | set(gold_struct)) if (pred_struct or gold_struct) else 1.0

    # Hardness
    hardness = classify_sql_hardness(extract_sql_metadata(gold_sql))

    results.append({
        'instance_id': query_id,
        'exact_match': exact_match,
        'jaccard_score': jaccard,
        'hardness': hardness
    })

# ----------------------------
# Results
# ----------------------------
results_df = pd.DataFrame(results)

print(f"✅ Exact Match Score: {results_df['exact_match'].mean():.2%}")
print(f"✅ Avg Jaccard Structural Score: {results_df['jaccard_score'].mean():.2%}")

print("\n📊 Count of Queries by Hardness:")
print(results_df['hardness'].value_counts().sort_index())

print("\n📊 Exact Match by Hardness:")
print(results_df.groupby('hardness')['exact_match'].mean())

print("\n📊 Jaccard Score by Hardness:")
print(results_df.groupby('hardness')['jaccard_score'].mean())

✅ Exact Match Score: 27.14%
✅ Avg Jaccard Structural Score: 71.21%

📊 Count of Queries by Hardness:
hardness
easy      1220
extra     2081
hard      1403
medium    2426
Name: count, dtype: int64

📊 Exact Match by Hardness:
hardness
easy      0.568852
extra     0.219125
hard      0.144690
medium    0.239901
Name: exact_match, dtype: float64

📊 Jaccard Score by Hardness:
hardness
easy      0.799443
extra     0.692704
hard      0.658208
medium    0.715931
Name: jaccard_score, dtype: float64


### spider 2.0 ablation starcoder 

In [216]:
# Load ground truth SQLs
df1 = pd.read_parquet('spider.parquet')

# Load predictions
df = pd.read_csv('refined_spider_queries_cleaned_basic_final_final.csv')

df2 = pd.merge(df, df1, left_on=['nl_query', 'db'], right_on=['question', 'db_id'], how='inner')
df.dropna(inplace=True)
df2.shape

results = []

for _, row in df2.iterrows():
    query_id = row['instance_id']
    pred_sql = row['refined_sql_cleaned']
    gold_sql = row['query']

    # Structure match
    pred_struct = normalize_structure(pred_sql)
    gold_struct = normalize_structure(gold_sql)

    exact_match = int(pred_struct == gold_struct)
    jaccard = len(set(pred_struct) & set(gold_struct)) / len(set(pred_struct) | set(gold_struct)) if (pred_struct or gold_struct) else 1.0

    # Hardness
    hardness = classify_sql_hardness(extract_sql_metadata(gold_sql))

    results.append({
        'instance_id': query_id,
        'exact_match': exact_match,
        'jaccard_score': jaccard,
        'hardness': hardness
    })

# ----------------------------
# Results
# ----------------------------
results_df = pd.DataFrame(results)

print(f"✅ Exact Match Score: {results_df['exact_match'].mean():.2%}")
print(f"✅ Avg Jaccard Structural Score: {results_df['jaccard_score'].mean():.2%}")

print("\n📊 Count of Queries by Hardness:")
print(results_df['hardness'].value_counts().sort_index())

print("\n📊 Exact Match by Hardness:")
print(results_df.groupby('hardness')['exact_match'].mean())

print("\n📊 Jaccard Score by Hardness:")
print(results_df.groupby('hardness')['jaccard_score'].mean())

✅ Exact Match Score: 18.67%
✅ Avg Jaccard Structural Score: 67.22%

📊 Count of Queries by Hardness:
hardness
easy      1220
extra     2081
hard      1403
medium    2426
Name: count, dtype: int64

📊 Exact Match by Hardness:
hardness
easy      0.304918
extra     0.190774
hard      0.114041
medium    0.165705
Name: exact_match, dtype: float64

📊 Jaccard Score by Hardness:
hardness
easy      0.698015
extra     0.680701
hard      0.636588
medium    0.672389
Name: jaccard_score, dtype: float64


### spider 2.0 vanna_generated

In [231]:
# Load ground truth SQLs
ground_truth_dir = 'spider2_lite_gold/sql'
ground_truth_queries = {}

for file_path in glob.glob(os.path.join(ground_truth_dir, '*.sql')):
    query_id = os.path.splitext(os.path.basename(file_path))[0]
    with open(file_path, 'r', encoding='utf-8') as f:
        ground_truth_queries[query_id] = f.read().strip()

# Load predictions
df = pd.read_csv('vanna_generated_queries.csv')

# Evaluate
results = []

for _, row in df.iterrows():
    query_id = row['instance_id']
    pred_sql = row['vanna_sql']
    gold_sql = ground_truth_queries.get(query_id, '')

    # Structure match
    pred_struct = normalize_structure(pred_sql)
    gold_struct = normalize_structure(gold_sql)

    exact_match = int(pred_struct == gold_struct)
    jaccard = len(set(pred_struct) & set(gold_struct)) / len(set(pred_struct) | set(gold_struct)) if (pred_struct or gold_struct) else 1.0

    # Hardness
    hardness = classify_sql_hardness(extract_sql_metadata(gold_sql))

    results.append({
        'instance_id': query_id,
        'exact_match': exact_match,
        'jaccard_score': jaccard,
        'hardness': hardness
    })

# ----------------------------
# Results
# ----------------------------
results_df = pd.DataFrame(results)

print(f"✅ Exact Match Score: {results_df['exact_match'].mean():.2%}")
print(f"✅ Avg Jaccard Structural Score: {results_df['jaccard_score'].mean():.2%}")

print("\n📊 Count of Queries by Hardness:")
print(results_df['hardness'].value_counts().sort_index())

print("\n📊 Exact Match by Hardness:")
print(results_df.groupby('hardness')['exact_match'].mean())

print("\n📊 Jaccard Score by Hardness:")
print(results_df.groupby('hardness')['jaccard_score'].mean())


✅ Exact Match Score: 33.09%
✅ Avg Jaccard Structural Score: 36.28%

📊 Count of Queries by Hardness:
hardness
easy      291
extra     253
hard        2
medium      1
Name: count, dtype: int64

📊 Exact Match by Hardness:
hardness
easy      0.563574
extra     0.067194
hard      0.000000
medium    0.000000
Name: exact_match, dtype: float64

📊 Jaccard Score by Hardness:
hardness
easy      0.563574
extra     0.136095
hard      0.000000
medium    0.000000
Name: jaccard_score, dtype: float64


## geosql

In [218]:
df1 = pd.read_csv('refined_geo_queries_cleaned_basic_final.csv')

In [219]:
df1.head()

,instance_id,nl_query,generated_sql,refined_sql,refined_sql_cleaned,final_query
0,SELECT state.area FROM state WHERE state.state...,what is the size of texas,NaN,NaN,NaN,SELECT state.area FROM state WHERE state.state...
1,SELECT city.city_name FROM city WHERE city.sta...,what is the biggest city in nebraska,SELECT city_name\nFROM cities\nWHERE state = '...,SELECT city_name\nFROM cities\nWHERE state = '...,SELECT city_name FROM cities WHERE state = 'Ne...,SELECT city_name FROM city WHERE state_name = ...
2,SELECT city.population FROM city WHERE city.ci...,how many people live in chicago,SELECT COUNT(*) \nFROM population \nWHERE city...,SELECT COUNT(*) \nFROM population \nWHERE city...,SELECT COUNT(*) FROM population WHERE city = '...,SELECT population FROM city WHERE city_name = ...
3,SELECT state.population FROM state WHERE state...,how many people live in new mexico,CREATE TABLE addresses (\n address_id numbe...,CREATE TABLE addresses (\n address_id numbe...,"CREATE TABLE addresses ( address_id number, li...","SELECT state_name, population FROM state WHERE..."
4,SELECT city.city_name FROM city WHERE city.sta...,what are the major cities in texas,"select the name of each city from the ""city"" t...",SELECT city_name\nFROM city\nORDER BY populati...,SELECT city_name FROM city ORDER BY population...,SELECT city_name FROM city WHERE state_name='t...


In [220]:
def extract_select_only(query):
    """
    Remove any CREATE or INSERT statements and return only the final SELECT query.
    Supports multi-statement SQL inputs.
    """
    # Split on semicolons to separate multiple statements
    statements = re.split(r';\s*', query)
    
    # Return the first SELECT statement found
    for stmt in statements:
        if stmt.strip().lower().startswith("select"):
            return stmt.strip()
    
    # If no SELECT found, return empty string
    return ""

In [221]:
df1['final_query'] = df1['final_query'].apply(extract_select_only)

In [222]:
results = []

for _, row in df1.iterrows():
    query_id = row['instance_id']
    pred_sql = row['final_query']
    gold_sql = row['instance_id']

    # Structure match
    pred_struct = normalize_structure(pred_sql)
    gold_struct = normalize_structure(gold_sql)

    exact_match = int(pred_struct == gold_struct)
    jaccard = len(set(pred_struct) & set(gold_struct)) / len(set(pred_struct) | set(gold_struct)) if (pred_struct or gold_struct) else 1.0

    # Hardness
    hardness = classify_sql_hardness(extract_sql_metadata(gold_sql))

    results.append({
        'instance_id': query_id,
        'exact_match': exact_match,
        'jaccard_score': jaccard,
        'hardness': hardness
    })

# ----------------------------
# Results
# ----------------------------
results_df = pd.DataFrame(results)

print(f"✅ Exact Match Score: {results_df['exact_match'].mean():.2%}")
print(f"✅ Avg Jaccard Structural Score: {results_df['jaccard_score'].mean():.2%}")

print("\n📊 Count of Queries by Hardness:")
print(results_df['hardness'].value_counts().sort_index())

print("\n📊 Exact Match by Hardness:")
print(results_df.groupby('hardness')['exact_match'].mean())

print("\n📊 Jaccard Score by Hardness:")
print(results_df.groupby('hardness')['jaccard_score'].mean())

✅ Exact Match Score: 72.64%
✅ Avg Jaccard Structural Score: 86.37%

📊 Count of Queries by Hardness:
hardness
easy      195
extra     128
hard      102
medium    105
Name: count, dtype: int64

📊 Exact Match by Hardness:
hardness
easy      0.876923
extra     0.531250
hard      0.529412
medium    0.876190
Name: exact_match, dtype: float64

📊 Jaccard Score by Hardness:
hardness
easy      0.942308
extra     0.785733
hard      0.727778
medium    0.944762
Name: jaccard_score, dtype: float64


In [223]:
import pyarrow as pa
import pandas as pd

with pa.memory_map("/Users/swapster/Desktop/Text2SQL/geosql/geoquery_nq_train_with_answer.hf/dataset.arrow", "rb") as source:
    reader = pa.ipc.RecordBatchStreamReader(source)
    table = reader.read_all()

df = table.to_pandas()
df.head()

,question,answer,table_names,tables,source,target
0,what is the size of texas,"{""columns"":[""area""],""index"":[0],""data"":[[26680...",[state],"[{""columns"":[""state_name"",""population"",""area"",...",what is the size of texas <table_name> : state...,col : area row 1 : 266807
1,what is the biggest city in nebraska,"{""columns"":[""city_name""],""index"":[0],""data"":[[...",[city],"[{""columns"":[""city_name"",""population"",""country...",what is the biggest city in nebraska <table_na...,col : city_name row 1 : omaha
2,how many people live in chicago,"{""columns"":[""population""],""index"":[0],""data"":[...",[city],"[{""columns"":[""city_name"",""population"",""country...",how many people live in chicago <table_name> :...,col : population row 1 : 3005172
3,how many people live in new mexico,"{""columns"":[""population""],""index"":[0],""data"":[...",[state],"[{""columns"":[""state_name"",""population"",""area"",...",how many people live in new mexico <table_name...,col : population row 1 : 1303000
4,what are the major cities in texas,"{""columns"":[""city_name""],""index"":[0,1,2,3,4,5,...",[city],"[{""columns"":[""city_name"",""population"",""country...",what are the major cities in texas <table_name...,col : city_name row 1 : houston row 2 : dallas...


In [224]:
df = df.merge(df1, left_on=['question'], right_on=['nl_query'], how='inner')
df.head(10)

,question,answer,table_names,tables,source,target,instance_id,nl_query,generated_sql,refined_sql,refined_sql_cleaned,final_query
0,what is the size of texas,"{""columns"":[""area""],""index"":[0],""data"":[[26680...",[state],"[{""columns"":[""state_name"",""population"",""area"",...",what is the size of texas <table_name> : state...,col : area row 1 : 266807,SELECT state.area FROM state WHERE state.state...,what is the size of texas,NaN,NaN,NaN,SELECT state.area FROM state WHERE state.state...
1,what is the biggest city in nebraska,"{""columns"":[""city_name""],""index"":[0],""data"":[[...",[city],"[{""columns"":[""city_name"",""population"",""country...",what is the biggest city in nebraska <table_na...,col : city_name row 1 : omaha,SELECT city.city_name FROM city WHERE city.sta...,what is the biggest city in nebraska,SELECT city_name\nFROM cities\nWHERE state = '...,SELECT city_name\nFROM cities\nWHERE state = '...,SELECT city_name FROM cities WHERE state = 'Ne...,SELECT city_name FROM city WHERE state_name = ...
2,how many people live in chicago,"{""columns"":[""population""],""index"":[0],""data"":[...",[city],"[{""columns"":[""city_name"",""population"",""country...",how many people live in chicago <table_name> :...,col : population row 1 : 3005172,SELECT city.population FROM city WHERE city.ci...,how many people live in chicago,SELECT COUNT(*) \nFROM population \nWHERE city...,SELECT COUNT(*) \nFROM population \nWHERE city...,SELECT COUNT(*) FROM population WHERE city = '...,SELECT population FROM city WHERE city_name = ...
3,how many people live in new mexico,"{""columns"":[""population""],""index"":[0],""data"":[...",[state],"[{""columns"":[""state_name"",""population"",""area"",...",how many people live in new mexico <table_name...,col : population row 1 : 1303000,SELECT state.population FROM state WHERE state...,how many people live in new mexico,CREATE TABLE addresses (\n address_id numbe...,CREATE TABLE addresses (\n address_id numbe...,"CREATE TABLE addresses ( address_id number, li...","SELECT state_name, population FROM state WHERE..."
4,what are the major cities in texas,"{""columns"":[""city_name""],""index"":[0,1,2,3,4,5,...",[city],"[{""columns"":[""city_name"",""population"",""country...",what are the major cities in texas <table_name...,col : city_name row 1 : houston row 2 : dallas...,SELECT city.city_name FROM city WHERE city.sta...,what are the major cities in texas,"select the name of each city from the ""city"" t...",SELECT city_name\nFROM city\nORDER BY populati...,SELECT city_name FROM city ORDER BY population...,SELECT city_name FROM city WHERE state_name='t...
5,which states have a major city named austin ?,"{""columns"":[""state_name""],""index"":[0],""data"":[...",[city],"[{""columns"":[""city_name"",""population"",""country...",which states have a major city named austin ? ...,col : state_name row 1 : texas,SELECT city.state_name FROM city WHERE city.ci...,which states have a major city named austin ?,SELECT state_name\nFROM geography_table\nWHERE...,SELECT state_name\nFROM geography_table\nWHERE...,SELECT state_name FROM geography_table WHERE c...,SELECT state_name FROM geography_table WHERE c...
6,which state is kalamazoo in,"{""columns"":[""state_name""],""index"":[0],""data"":[...",[city],"[{""columns"":[""city_name"",""population"",""country...",which state is kalamazoo in <table_name> : cit...,col : state_name row 1 : michigan,SELECT city.state_name FROM city WHERE city.ci...,which state is kalamazoo in,SELECT state FROM geography WHERE city = 'Kala...,SELECT state FROM geography WHERE city = 'Kala...,SELECT state FROM geography WHERE city = 'Kala...,SELECT state FROM geography WHERE city = 'Kala...
7,how many states border hawaii,"{""columns"":[""count(border_info.border)""],""inde...",[border_info],"[{""columns"":[""state_name"",""border""],""index"":[0...",how many states border hawaii <table_name> : b...,col : count(border_info.border) row 1 : 0,SELECT count(border_info.border) FROM b

In [225]:
import pandas as pd
import json
import numpy as np
import re
from pandasql import sqldf
from collections import Counter

def force_alias_to_t(query):
    """
    Replace all table references (e.g., FROM state, state.area) with alias 't'.
    """
    table_refs = re.findall(r'\b(?:FROM|JOIN)\s+([a-zA-Z_][a-zA-Z0-9_]*)', query, flags=re.IGNORECASE)
    prefix_refs = re.findall(r'([a-zA-Z_][a-zA-Z0-9_]*)\.', query)
    all_candidates = set(table_refs + prefix_refs)

    for name in all_candidates:
        query = re.sub(rf'\bFROM\s+{name}\b', 'FROM t', query, flags=re.IGNORECASE)
        query = re.sub(rf'\bJOIN\s+{name}\b', 'JOIN t', query, flags=re.IGNORECASE)
        query = re.sub(rf'\b{name}\.', 't.', query, flags=re.IGNORECASE)
    
    return query

def get_all_referenced_columns(query):
    """
    Extract column names in the form of 't.column_name' from SQL query.
    """
    return set(re.findall(r't\.([a-zA-Z_][a-zA-Z0-9_]*)', query))

def evaluate_execution_accuracy(df):
    correct = 0
    total = len(df)
    failed_queries = []
    missing_column_stats = Counter()

    for i, row in df.iterrows():
        try:
            # --- Parse table JSON ---
            table_str = row['tables']
            if isinstance(table_str, (list, tuple, pd.Series, np.ndarray)):
                table_str = table_str[0]
            table_json = json.loads(table_str)
            table_df = pd.DataFrame(table_json['data'], columns=table_json['columns'])

            # --- Prepare and rewrite query ---
            query = force_alias_to_t(row['final_query'])

            # --- Check for missing columns ---
            query_cols = get_all_referenced_columns(query)
            missing_cols = query_cols - set(table_df.columns)
            if missing_cols:
                failed_queries.append((i, f"Missing columns in table: {missing_cols}"))
                missing_column_stats.update(missing_cols)
                continue

            # --- Execute SQL using alias 't' ---
            pysqldf = lambda q: sqldf(q, {'t': table_df})
            pred_result_df = pysqldf(query)

            # --- Parse ground truth answer ---
            answer_str = row['answer']
            if isinstance(answer_str, (list, tuple, pd.Series, np.ndarray)):
                answer_str = answer_str[0]
            answer_json = json.loads(answer_str)
            true_df = pd.DataFrame(answer_json['data'], columns=answer_json['columns'])

            # --- Compare results ---
            if pred_result_df.reset_index(drop=True).equals(true_df.reset_index(drop=True)):
                correct += 1
            else:
                failed_queries.append((i, "Mismatch"))

        except Exception as e:
            failed_queries.append((i, f"Execution error: {e}"))

    # --- Print final stats ---
    accuracy = correct / total if total > 0 else 0
    print(f"\n✅ Execution Accuracy: {accuracy:.2%} ({correct}/{total})")

    if failed_queries:
        print("\n❌ Failed Queries Summary:")
        for idx, reason in failed_queries:
            print(f" - Row {idx}: {reason}")

    if missing_column_stats:
        print("\n📉 Most Common Missing Columns:")
        for col, count in missing_column_stats.most_common(10):
            print(f" - {col}: {count} times")

    return accuracy


In [226]:
accuracy = evaluate_execution_accuracy(df)


✅ Execution Accuracy: 17.04% (91/534)

❌ Failed Queries Summary:
 - Row 2: Mismatch
 - Row 3: Mismatch
 - Row 5: Mismatch
 - Row 6: Execution error: (sqlite3.OperationalError) no such column: state
[SQL: SELECT state FROM t WHERE city = 'Kalamazoo' AND state = 'CA']
(Background on this error at: https://sqlalche.me/e/20/e3q8)
 - Row 7: Mismatch
 - Row 9: Execution error: (sqlite3.OperationalError) no such column: name
[SQL: SELECT name FROM t WHERE area=(SELECT max(area) FROM t)]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
 - Row 10: Mismatch
 - Row 11: Execution error: (sqlite3.OperationalError) no such column: state
[SQL: SELECT state FROM t WHERE country = 'United States' AND city = 'Austin' AND state = 'TX']
(Background on this error at: https://sqlalche.me/e/20/e3q8)
 - Row 12: Execution error: (sqlite3.OperationalError) no such column: name
[SQL: SELECT name FROM t WHERE state_id IN (SELECT state_id FROM t WHERE border_state = 'Texas') ORDER BY population DESC L

### spider 2.0 ablation qwen 

In [227]:
results = []

for _, row in df1.iterrows():
    query_id = row['instance_id']
    pred_sql = row['generated_sql']
    gold_sql = row['instance_id']

    # Structure match
    pred_struct = normalize_structure(pred_sql)
    gold_struct = normalize_structure(gold_sql)

    exact_match = int(pred_struct == gold_struct)
    jaccard = len(set(pred_struct) & set(gold_struct)) / len(set(pred_struct) | set(gold_struct)) if (pred_struct or gold_struct) else 1.0

    # Hardness
    hardness = classify_sql_hardness(extract_sql_metadata(gold_sql))

    results.append({
        'instance_id': query_id,
        'exact_match': exact_match,
        'jaccard_score': jaccard,
        'hardness': hardness
    })

# ----------------------------
# Results
# ----------------------------
results_df = pd.DataFrame(results)

print(f"✅ Exact Match Score: {results_df['exact_match'].mean():.2%}")
print(f"✅ Avg Jaccard Structural Score: {results_df['jaccard_score'].mean():.2%}")

print("\n📊 Count of Queries by Hardness:")
print(results_df['hardness'].value_counts().sort_index())

print("\n📊 Exact Match by Hardness:")
print(results_df.groupby('hardness')['exact_match'].mean())

print("\n📊 Jaccard Score by Hardness:")
print(results_df.groupby('hardness')['jaccard_score'].mean())

✅ Exact Match Score: 53.40%
✅ Avg Jaccard Structural Score: 73.57%

📊 Count of Queries by Hardness:
hardness
easy      195
extra     128
hard      102
medium    105
Name: count, dtype: int64

📊 Exact Match by Hardness:
hardness
easy      0.697436
extra     0.359375
hard      0.313725
medium    0.657143
Name: exact_match, dtype: float64

📊 Jaccard Score by Hardness:
hardness
easy      0.860385
extra     0.615123
hard      0.568581
medium    0.813651
Name: jaccard_score, dtype: float64


### spider 2.0 ablation starcoder 

In [230]:
results = []

for _, row in df1.iterrows():
    query_id = row['instance_id']
    pred_sql = row['refined_sql_cleaned']
    gold_sql = row['instance_id']

    # Structure match
    pred_struct = normalize_structure(pred_sql)
    gold_struct = normalize_structure(gold_sql)

    exact_match = int(pred_struct == gold_struct)
    jaccard = len(set(pred_struct) & set(gold_struct)) / len(set(pred_struct) | set(gold_struct)) if (pred_struct or gold_struct) else 1.0

    # Hardness
    hardness = classify_sql_hardness(extract_sql_metadata(gold_sql))

    results.append({
        'instance_id': query_id,
        'exact_match': exact_match,
        'jaccard_score': jaccard,
        'hardness': hardness
    })

# ----------------------------
# Results
# ----------------------------
results_df = pd.DataFrame(results)

print(f"✅ Exact Match Score: {results_df['exact_match'].mean():.2%}")
print(f"✅ Avg Jaccard Structural Score: {results_df['jaccard_score'].mean():.2%}")

print("\n📊 Count of Queries by Hardness:")
print(results_df['hardness'].value_counts().sort_index())

print("\n📊 Exact Match by Hardness:")
print(results_df.groupby('hardness')['exact_match'].mean())

print("\n📊 Jaccard Score by Hardness:")
print(results_df.groupby('hardness')['jaccard_score'].mean())

✅ Exact Match Score: 52.26%
✅ Avg Jaccard Structural Score: 76.16%

📊 Count of Queries by Hardness:
hardness
easy      195
extra     128
hard      102
medium    105
Name: count, dtype: int64

📊 Exact Match by Hardness:
hardness
easy      0.666667
extra     0.343750
hard      0.343137
medium    0.647619
Name: exact_match, dtype: float64

📊 Jaccard Score by Hardness:
hardness
easy      0.831581
extra     0.688821
hard      0.660247
medium    0.819002
Name: jaccard_score, dtype: float64
